In [128]:
import numpy as np
import pandas as pd
import seaborn as sns
import cufflinks as cf
import plotly.express as px
import chart_studio.plotly as py
%matplotlib inline

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
cf.go_offline()


def scale_axis(data: pd.DataFrame, scale:dict) -> pd.DataFrame:
    # scale = {"Sun": 10e6, "Mercury": 10e8, "Venus":10e8, "Earth":10e8, "Mars":10e8, "Jupiter":10e10, "Saturn":10e10, "Uranus":10e10, "Neptune": 10e10}
    data = data.copy()
    new_list = []
    for key in data["name"].to_list():
        new_list.append(scale[key])
    data["scale"] = new_list
    data["x"] = data["x"] / data["scale"]
    data["y"] = data["y"] / data["scale"]
    data["z"] = data["z"] / data["scale"]
    return data

In [127]:
new = scale_axis(position_history)
new.tail(30)

,index,time,x,y,z,name,radius,mass,scale
2890,361,2022:05:14,-217.115090,-95.616470,-38.012116,Mars,58.219413,6.390000e+23,1.000000e+09
2891,361,2022:05:14,6.618293,11.102365,4.301388,Saturn,241.313075,5.683000e+26,1.000000e+11
2892,361,2022:05:14,23.187207,-17.117675,-7.825241,Uranus,159.254513,8.681000e+25,1.000000e+11
2893,361,2022:05:14,26.725978,-33.310592,-14.298041,Neptune,160.068735,1.024000e+26,1.000000e+11
2894,361,2022:05:14,-57.605254,-0.181063,5.845969,Mercury,49.393319,3.285000e+23,1.000000e+09
2895,361,2022:05:14,2.134799,6.696331,2.817999,Jupiter,264.406883,1.898000e+27,1.000000e+11
2896,362,2022:05:15,6.610550,11.105983,4.303215,Saturn,241.313075,5.683000e+26,1.000000e+11
2897,362,2022:05:15,26.729719,-33.307943,-14.297050,Neptune,160.068735,1.024000e+26,1.000000e+11
2898,362,2022:05:15,2.123814,6.699648,2.819689,Jupiter,264.406883,1.898000e+27,1.000000e+11
2899,362,2022:05:15,11.416520,12.083497,2.386305,Sun,26388.254963,1.988470e+30,1.000000e+07


In [2]:
planets_data = pd.read_csv("planets_data.csv", index_col=0)
planets_data = planets_data[planets_data.name != "Venus"]
number_of_planets = planets_data.shape[0]
planets_data

,name,mass,color,radius
0,Sun,1.988470e+30,yellow,696340000.0
1,Mercury,3.285000e+23,orange,2439.7
3,Earth,6.046000e+24,blue,6371.0
4,Mars,6.390000e+23,red,3389.5
5,Jupiter,1.898000e+27,orange,69911.0
6,Saturn,5.683000e+26,brown,58232.0
7,Uranus,8.681000e+25,purple,25362.0
8,Neptune,1.024000e+26,cyan,25622.0


In [168]:
import datetime
scale = {"Sun": 10e6, "Mercury": 10e8, "Venus":10e8, "Earth":10e8, "Mars":3*10e8,
         "Jupiter":10e9, "Saturn":10e9, "Uranus":10e9, "Neptune": 10e9}

position_history = pd.read_csv("position_history.csv", index_col=0)
position_history["time"] = pd.to_datetime(position_history["time"]).astype("datetime64[D]")
position_history = scale_axis(position_history, scale=scale)
position_history = position_history[position_history.name != "Venus"]
position_history["radius"] = np.log(position_history["radius"])
position_history = position_history.sort_values("time")

start_time = min(position_history["time"])
end_time = max(position_history["time"])

position_history["time"] = position_history["time"].dt.strftime("%Y:%m:%d")

x_limit = position_history["x"].abs().max()
y_limit = position_history["y"].abs().max()
z_limit = position_history["z"].abs().max()

position_history = position_history.reset_index()
position_history.head(number_of_planets)

,index,time,x,y,z,name,radius,mass,scale
0,0,2021:05:18,0.000000,0.000000,0.000000,Sun,20.361349,1.988470e+30,1.000000e+07
1,0,2021:05:18,59.988746,36.257537,15.009653,Mars,8.128438,6.390000e+23,3.000000e+09
2,0,2021:05:18,-37.444347,28.019681,18.849332,Mercury,7.799630,3.285000e+23,1.000000e+09
3,0,2021:05:18,253.568391,-342.429526,-146.456315,Neptune,10.151207,1.024000e+26,1.000000e+10
4,0,2021:05:18,92.212127,95.248964,35.379896,Saturn,10.972190,5.683000e+26,1.000000e+10
5,0,2021:05:18,218.113695,-185.052566,-84.133842,Uranus,10.141007,8.681000e+25,1.000000e+10
6,0,2021:05:18,55.844885,45.657270,18.206061,Jupiter,11.154978,1.898000e+27,1.000000e+10
7,0,2021:05:18,-136.014384,53.750515,23.307348,Earth,8.759512,6.046000e+24,1.000000e+09


In [151]:
steps = 50
time_step = ((end_time - start_time) / steps).days

downsampled_position_history = position_history[0:number_of_planets]
for index in range(number_of_planets*time_step, position_history.shape[0], number_of_planets*time_step):
    downsampled_position_history = downsampled_position_history.append(position_history[index:index+number_of_planets])

pd.options.display.max_rows = 300
downsampled_position_history.head(20)

,index,time,x,y,z,name,radius,mass,scale
0,0,2021:05:18,0.000000,0.000000,0.000000,Sun,20.361349,1.988470e+30,1.000000e+07
1,0,2021:05:18,59.988746,36.257537,15.009653,Mars,8.128438,6.390000e+23,3.000000e+09
2,0,2021:05:18,-37.444347,28.019681,18.849332,Mercury,7.799630,3.285000e+23,1.000000e+09
3,0,2021:05:18,253.568391,-342.429526,-146.456315,Neptune,10.151207,1.024000e+26,1.000000e+10
4,0,2021:05:18,92.212127,95.248964,35.379896,Saturn,10.972190,5.683000e+26,1.000000e+10
5,0,2021:05:18,218.113695,-185.052566,-84.133842,Uranus,10.141007,8.681000e+25,1.000000e+10
6,0,2021:05:18,55.844885,45.657270,18.206061,Jupiter,11.154978,1.898000e+27,1.000000e+10
7,0,2021:05:18,-136.014384,53.750515,23.307348,Earth,8.759512,6.046000e+24,1.000000e+09
56,7,2021:05:25,55.308902,46.229893,18.464581,Jupiter,11.154978,1.898000e+27,1.000000e+10
57,7,2021:05:25,91.747670,95.603478,35.546274,Saturn,10.972190,5.683000e+26,1.000000e+10


In [152]:
steps = []
slider_range = int(downsampled_position_history.shape[0]/8)
for i in range(slider_range):
    step = dict(
        method="update",
        visible=False,
    )
    steps.append(step)

for step in steps[::3]:
    step["visible"]=True
    

sliders = [dict(
    steps=steps
)]

In [178]:
fig = px.scatter_3d(downsampled_position_history, x="x", y="y", z="z", color="name", size="radius",
                    animation_frame="index", animation_group="name",
                    template="plotly_dark",
                    color_discrete_sequence=["orange", "crimson", "cornflowerblue", "cyan", 
                                             "khaki", "lightcyan", "chocolate", "green"],
                   )

# fig.update_layout(
#     scene = dict(
#         xaxis = dict(nticks=2, range=[-x_limit, x_limit],),
#         yaxis = dict(nticks=2, range=[-y_limit, y_limit],),
#         zaxis = dict(nticks=2, range=[-z_limit, z_limit],),),
#     width=700,
#     margin=dict(r=20, l=10, b=10, t=10))
fig.update_layout(scene_aspectmode='cube', 
                  scene = dict(xaxis = dict(nticks=3, range=[-x_limit, x_limit],),
                               yaxis = dict(nticks=3, range=[-y_limit, y_limit],),
                               zaxis = dict(nticks=3, range=[-z_limit, z_limit],),
                              ),
                  width=700,)
# fig.update_layout(sliders=sliders)
fig